In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
df = pd.read_csv("data/www2017/wikihowURLID-wikihowURL-youtubeURL", sep="\t", names=["fid", "wikihow", "youtube"])
df['query'] = [i.split("/")[-1].replace("-"," ") for i in df.wikihow.tolist()]
df['query'] = df['query'].str.lower()
df['label'] = [i.split("/")[-1] for i in df.youtube.tolist()]


In [4]:

def read_youtube_video_json():
    vids, titles, descs, tags, comments = [], [], [], [], []
    with open("data/www2017/uniq-youtube-video.json", encoding="utf-8") as file:
        for l in file:
            y = json.loads(l)
            vids.append(y['id'])
            titles.append(y['title'])
            descs.append(y['description'])
            tags.append(' '.join(y['tags']))
            comments.append(' '.join([i['comment'] for i in y['comment']]))
    
    return pd.DataFrame({'vid':vids, 'title':titles, 'desc':descs, 'comment':comments, 'tag':tags})

df_video = read_youtube_video_json()

In [5]:

def get_wikihow_with_video():
    linkids = []
    with open("data/www2017/task-frame-have-video.json", encoding="utf-8") as file:
        for l in file:
            y = json.loads(l)
            linkids.append(int(y['activity']['linkid']))
    return linkids
wikihow_with_video = get_wikihow_with_video()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [i+" "+j+" "+k+" "+l for i, j, k, l in df_video[['title', 'desc', 'tag', 'comment']].values]
corpus = corpus + df['query'].tolist()
vectorizer = TfidfVectorizer(use_idf=False)
doc_vec = vectorizer.fit_transform(corpus)

In [7]:
query_vec = vectorizer.transform(df['query'])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
scores = cosine_similarity(query_vec, doc_vec)

In [17]:
%%time

def getHitRatio(ranklist, gtDoc, K):
    for i in range(K):
        item = ranklist[i]
        if item == gtDoc:
            return 1
    return 0

def getMRR(ranklist, gtDoc):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtDoc:
            return 1.0 / (i+1)
    return 0

def evaluate(scores):
    hr1s, hr3s, hr5s, hr10s, mrrs = [], [], [], [], []
    for score, gtDoc in zip(scores, df.label):
        ranklist = {i:j for i,j in zip(df_video.vid, score)}
        ranklist = sorted(ranklist, key=ranklist.get, reverse=True)
        hr1 = getHitRatio(ranklist, gtDoc, 1)
        hr3 = getHitRatio(ranklist, gtDoc, 3)
        hr5 = getHitRatio(ranklist, gtDoc, 5)
        hr10 = getHitRatio(ranklist, gtDoc, 10)
        mrr = getMRR(ranklist, gtDoc)
        hr1s.append(hr1)
        hr3s.append(hr3)
        hr5s.append(hr5)
        hr10s.append(hr10)
        mrrs.append(mrr)
    print(np.mean(mrrs))
evaluate(scores)

0.3480687384119908
CPU times: user 3min 26s, sys: 563 ms, total: 3min 27s
Wall time: 3min 27s


In [9]:
from gensim.models import KeyedVectors

/Users/jarana/anaconda3/envs/recsys2019/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
wv_from_text = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
# def w2v_expansion():
res = []
for q in df['query']:
    print(q)
    expand = []
    for term in q.split():
        try:
            sim_terms = wv_from_text.most_similar(positive=[term], topn=2)
        except:
            continue
        expand.extend([i[0] for i in sim_terms])
    expand_query = ' '.join(expand)
    res.append(q+" "+expand_query)
    break
#     return res
# df['w2v_query'] = w2v_expansion()

patrol recent changes on wikihow


In [11]:
terms = []
for q in df['query']:
    terms.extend(q.split(" "))

In [16]:
terms = list(set(terms))

In [13]:
from tqdm.autonotebook import tqdm

/Users/jarana/anaconda3/envs/recsys2019/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
import collections
term2simterm = collections.defaultdict(str)
for term in tqdm(terms, total=len(terms)):
    try:
        sim_terms = wv_from_text.most_similar(positive=[term], topn=2)
        term2simterm[term] = " ".join([i[0] for i in sim_terms])
    except:
        continue

In [29]:
term2simterm['tart']

'fruity tangy'